# รายวิชา: ข้อมูลขนาดใหญ่ (Big Data) — Week 2: File Formats & Modern Storage

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/witsarutsarai12-Academic/128-356-Big-Data/blob/main/BigData_Week2_FileFormats_and_Storage.ipynb)

**Learning outcomes:**
- อธิบายความต่างระหว่าง File System vs Database และ Data Lake vs Lakehouse
- แยกแยะ Row-format vs Columnar-format พร้อมเหตุผลเชิงประสิทธิภาพ (compression, column pruning, predicate pushdown)
- แปลง CSV → Parquet และ benchmark ขนาด/เวลาอ่านเขียนได้
- ใช้ DuckDB/Pandas อ่าน Parquet และออกแบบ mini-lab เพื่อสาธิตประสิทธิภาพ



## Part 0: เตรียมสภาพแวดล้อม
- รันบน Google Colab หรือเครื่องที่มี RAM ≥ 8GB
- ไลบรารี: `pandas`, `numpy`, `pyarrow`/`fastparquet`, `duckdb`
- ถ้าเพิ่งเปิด Colab ใหม่ ให้ติดตั้งด้านล่างก่อน



In [ ]:
# ตรวจสอบเวอร์ชัน Python
import sys
print(sys.version)



3.12.12 (main, Oct  9 2025, 11:07:00) [Clang 16.0.0 (clang-1600.0.26.6)]


In [ ]:
# ติดตั้งไลบรารีที่ต้องใช้ (รันครั้งเดียวพอ)
%pip -q install pandas pyarrow fastparquet duckdb




[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Part 1: The Computer & Data — Mental Model
- **Storage (SSD/HDD)** = ตู้เย็น/ห้องเก็บของ — จุเยอะ, ช้า, ปิดไฟของยังอยู่
- **RAM** = โต๊ะเขียง — พื้นที่น้อยแต่เร็ว ต้องยกของจากตู้เย็นมาก่อนถึงจะปรุงได้
- **CPU** = พ่อครัว — ลงมือหั่น/ผัด
- **กฎเหล็ก**: ทุกอย่างต้องถูกโหลดจาก Storage → RAM → CPU เสมอ

![Kitchen Analogy](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/kitchen_analogy.png?raw=1)



### Key Terms — Storage & Filesystems
- **Block/Cluster**: หน่วยเล็กสุดที่ disk ใช้เก็บข้อมูล (เช่น 4KB)
- **Inode/Metadata**: สมุดทะเบียนของไฟล์ (สิทธิ์, เวลา, ขนาด, ตำแหน่ง block)
- **Page Cache**: RAM ที่ OS แอบเก็บไฟล์ที่เพิ่งอ่าน เพื่ออ่านซ้ำเร็วขึ้น
- **Sequential vs Random I/O**: อ่านต่อเนื่องเร็วกว่าอ่านกระโดดเยอะ ๆ



### File System Internals
- ไฟล์ถูกหั่นเป็น **Block/Cluster** (ส่วนใหญ่ 4KB)
- ไฟล์เล็กมากกินทั้งบล็อก → **Slack space**
- ไฟล์ใหญ่ถูกกระจายหลายบล็อก → **Fragmentation**
- เมื่อมีไฟล์เล็กจำนวนมาก (small files problem) ระบบต้องวิ่งหา block หลายตำแหน่ง → ช้า



### File Read Path (เปิดไฟล์เกิดอะไรขึ้นบ้าง?)
1) `open()` → OS หา metadata (inode)
2) แปลง filename → block list
3) OS ดึง block จาก disk → RAM (page cache)
4) โปรแกรมอ่านจาก RAM (ไม่อ่านจาก disk ตรง ๆ)

![File Read Path](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/file_read_path.png?raw=1)

**Key insight:** ถ้าอ่านซ้ำ และยังอยู่ใน page cache → เร็วมาก



### Fragmentation & Sequential Read
- ถ้าไฟล์กระจายหลายตำแหน่ง → disk ต้อง “กระโดดหัวอ่าน” → ช้าลง
- ระบบไฟล์สมัยใหม่พยายามจัดให้ไฟล์ใหญ่ต่อเนื่องกัน แต่ไฟล์เล็กจำนวนมากทำให้ fragmentation เพิ่ม

![Disk Fragmentation](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/disk_fragmentation.png?raw=1)



### Distributed File Systems (HDFS / S3) — ภาพใหญ่ให้เห็น Block & Metadata
- **HDFS**: ค่าเริ่มต้น block ~128MB พร้อม **replication factor = 3** เพื่อกันเครื่องล่ม; NameNode เก็บ metadata ไว้ใน RAM จึงเกลียดไฟล์จิ๋วจำนวนมาก
- **การเขียนไฟล์**: client buffer ไว้จนเต็ม block แล้วค่อยส่งต่อ DataNode → ลด network round-trip
- **Object Storage (S3/MinIO)**: ไม่มี concept block แต่ยังเจอปัญหาไฟล์เล็ก เพราะต้องเปิด request ทีละไฟล์และ metadata lookup หนัก
- **ข้อแนะนำเชิงปฏิบัติ**: เล็งขนาดไฟล์ Parquet 128–512MB ต่อไฟล์ และจำนวน partition ให้พอดีกับเงื่อนไข query (เช่น วัน/เดือน ไม่ใช่ user_id)



### HDFS vs Object Storage (S3) — ต่างกันยังไง
| มิติ | HDFS | S3/Object Storage |
|---|---|---|
| หน่วยข้อมูล | block | object |
| Metadata | NameNode (RAM-heavy) | Service metadata |
| Latency | ต่ำกว่าใน cluster | สูงกว่าเล็กน้อย |
| Update/Append | ทำได้ (append จำกัด) | ปกติ replace ทั้ง object |

![HDFS vs S3](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/hdfs_vs_s3.png?raw=1)

**สรุป:** HDFS เหมาะกับ on-prem cluster; S3 เหมาะกับ cloud scale + durability



#### Small Files Problem: อาการ & วิธีแก้ (Deep Dive)

**อาการ (Symptoms)**:
- **NameNode / Metadata Service Overload**: สมมติเรามีไฟล์ 1TB ถ้าเก็บเป็น 1 ไฟล์ใหญ่ = 1 metadata entry. แต่ถ้าเก็บเป็น 1 ล้านไฟล์เล็ก (1MB/file) = 1 ล้าน entries ที่ต้องจำใน RAM ของ NameNode -> ระบบล่ม (OOM).
- **Slow List/Read**: การ list file 1 ล้านไฟล์กินเวลานานมาก และการอ่านข้อมูลต้องเสียเวลา "Open/Close" ไฟล์บ่อยกว่าเวลาอ่านเนื้อหาจริง

**Analogy (อุปมา)**:
- เหมือนไปซื้อของ 1,000 ชิ้น:
   - **Large File**: ได้ใบเสร็จยาว 1 ใบ (จัดการง่าย)
   - **Small Files**: ได้ใบเสร็จสั้นๆ 1,000 ใบ (จัดการยาก, หายง่าย, ตรวจสอบช้า)

![Small Files Bottleneck](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/small_files_bottleneck.png?raw=1)



### Real-World Use Case: Log Data Platform (Shopee/Lazada style)
**สถานการณ์**: มี User click logs เข้ามาวินาทีละ 100,000 events
- **ถ้า Save ทุกวินาที**: จะได้ไฟล์เล็ก ๆ จำนวนมหาศาล (Small Files Problem) ภายในไม่กี่ชั่วโมง
- **วิธีแก้**: ใช้ Buffer (Kafka/Memory) รอให้ครบ 5-10 นาที หรือครบ 500MB แล้วค่อยเขียนลง Storage เป็นไฟล์ใหญ่ 1 ไฟล์ (Batch Write)

### Scenario Exercise — เลือกกลยุทธ์จัดไฟล์
1) ข้อมูล log รายชั่วโมง 1MB ต่อไฟล์ ตลอดปี → จะ partition อย่างไร?  
2) ข้อมูลธุรกรรม 200GB/วัน ต้อง query รายวัน → ตั้งขนาดไฟล์เป้าหมายเท่าไร?  
3) ถ้าต้องอ่านแค่คอลัมน์ `status` กับ `date` เป็นหลัก → ควรเก็บแบบไหน?



<details>
<summary><b>💡 เฉลยและคำอธิบาย (Scenario Exercise) - คลิกเพื่อดูคำตอบ</b></summary>

1. **Log รายชั่วโมง 1MB/file**:
   - ❌ **ห้าม Partition รายชั่วโมง**: จะเกิดไฟล์ 1MB จำนวนปีละ 8,760 ไฟล์ (Small Files!)
   - ✅ **วิธีแก้**: เขียนลง temp folder แล้วใช้ process ทำ **Compaction** รวบเป็นรายวัน (Day Partition) หรือรายเดือน ให้ไฟล์ใหญ่ขึ้น (เช่น ~100MB+)

2. **Transaction 200GB/วัน**:
   - ✅ **ขนาดไฟล์เป้าหมาย**: ควรตัดที่ **128MB - 512MB** ต่อไฟล์
   - เหตุผล: เพื่อให้ Engine อ่านแบบ Parallel ได้ดีที่สุด (ถ้าไฟล์ใหญ่ 200GB ก้อนเดียว เครื่องเดียวอ่านตาย แต่ถ้าซอยเป็น 100 ไฟล์ 2GB ก็กระจายให้ 100 cores ช่วยกันอ่านได้)

3. **Query แค่ Status/Date**:
   - ✅ **ใช้ Parquet**: เพราะเป็น Columnar Format
   - ระบบจะอ่านแค่ 2 คอลัมน์ที่ขอ แล้วข้ามอีก 98 คอลัมน์ที่เหลือ (Column Pruning) → เร็วและประหยัดเงินค่า Read API

</details>


### Activity 1 — Small Files Problem (ลงมือเอง)
ให้ลองสร้างไฟล์เล็กจำนวนมาก แล้วดูเวลาที่ระบบใช้ในการ zip หรืออ่านรายชื่อไฟล์
- คาดหวัง: การจัดการไฟล์ 5,000 ไฟล์ขนาด 1KB มักช้ากว่าไฟล์เดียวขนาด 5MB
- ปรับจำนวนไฟล์แล้วสังเกตเวลา



In [5]:
# Demo: สร้างไฟล์จิ๋วจำนวนมาก vs ไฟล์เดียว แล้ววัดเวลา listdir / zip
import os, tempfile, time, shutil, pathlib

N_FILES = 2000   # ปรับจำนวนไฟล์เล็กๆ
SMALL_BYTES = 1024  # 1KB ต่อไฟล์

base = pathlib.Path(tempfile.mkdtemp())
small_dir = base / "small"
big_dir = base / "big"
small_dir.mkdir(); big_dir.mkdir()

# สร้างไฟล์เล็กจำนวนมาก
start = time.time()
for i in range(N_FILES):
    (small_dir / f"f_{i}.txt").write_bytes(b"a" * SMALL_BYTES)
small_elapsed = time.time() - start

# สร้างไฟล์ใหญ่ก้อนเดียวขนาดเท่ากัน
big_path = big_dir / "one_big.txt"
start = time.time()
big_path.write_bytes(b"a" * SMALL_BYTES * N_FILES)
big_elapsed = time.time() - start

print(f"สร้างไฟล์เล็ก {N_FILES} ไฟล์ ใช้เวลา {small_elapsed:.2f}s")
print(f"สร้างไฟล์ใหญ่ไฟล์เดียว ใช้เวลา {big_elapsed:.2f}s")

# วัดเวลา listdir (metadata lookup)
start = time.time(); len(list(small_dir.iterdir())); ls_small = time.time() - start
start = time.time(); len(list(big_dir.iterdir())); ls_big = time.time() - start
print(f"listdir โฟลเดอร์ไฟล์เล็ก ใช้ {ls_small*1000:.1f} ms; โฟลเดอร์ไฟล์ใหญ่ ใช้ {ls_big*1000:.1f} ms")

shutil.rmtree(base)



สร้างไฟล์เล็ก 2000 ไฟล์ ใช้เวลา 0.12s
สร้างไฟล์ใหญ่ไฟล์เดียว ใช้เวลา 0.00s
listdir โฟลเดอร์ไฟล์เล็ก ใช้ 6.6 ms; โฟลเดอร์ไฟล์ใหญ่ ใช้ 0.1 ms


### File System vs Database (ทำไมต้องมี DB?)
| มิติ | File (CSV/TXT) | Database (MySQL/Postgres) |
|---|---|---|
| ความน่าเชื่อถือ | เขียนไม่จบไฟดับ = ไฟล์พัง | **ACID** คุมความถูกต้อง |
| การค้นหา | ต้องอ่านทุกบรรทัด (scan) | มี **Index** กระโดดไปจุดที่ต้องการ |
| การแก้ไข | แก้ 1 บรรทัด = เขียนใหม่ทั้งไฟล์ | แก้เฉพาะ row ได้ |



### CSV Reality Check — จุดที่มักพัง
- **Delimiter/encoding ต่างกัน**: `,` vs `;` vs `	`, UTF-8 vs Windows-874 → อ่านเพี้ยน
- **Quoted fields**: ค่า string ที่มี comma / newline ต้องถูกล้อมด้วย `""`; ถ้าไม่ใส่ → column shift
- **Missing header / column drift**: ไฟล์จากคนละระบบอาจลำดับคอลัมน์ต่างกัน
- **Type drift**: ตัวเลขที่มี leading zero (`00123`) ถูกแปลงเป็น int → ศูนย์หาย
- แนวทาง: ระบุ `dtype` ชัด, ใช้ `quotechar`/`escapechar`, ตรวจ schema ก่อนใช้, ถ้าไฟล์ใหญ่ → พิจารณาแปลงเป็น Parquet


In [ ]:
# ตัวอย่างปัญหา CSV: comma ภายในข้อความ
import pandas as pd
from io import StringIO

raw = "id,name,comment
1,Alice,"Likes, commas"
2,Bob,Plain text
"
print(pd.read_csv(StringIO(raw)))
print('
กำหนด quotechar ถูกต้อง:')
print(pd.read_csv(StringIO(raw), quotechar='"'))



### File vs DB — ใช้ตอนไหน? (Comparison Strategy)

![File vs DB Decision](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/file_vs_db_decision.png?raw=1)

| Feature | File System (Parquet/CSV) | Database (MySQL/Postgres) |
|---|---|---|
| **Primary Goal** | Analytics / Batch Processing (อ่านเยอะ เขียนครั้งเดียว) | Transactional / Real-time Updates (อ่านเขียนตลอดเวลา) |
| **Cost** | ถูก (S3/HDFS) | แพง (Server Resource/License) |
| **Update** | ยาก (ต้องเขียนทับทั้งไฟล์) | ง่าย (Update ทีละ row ได้) |
| **ACID** | ไม่มี (ต้องใช้ Lakehouse ช่วย) | มีครบ (รับประกันความถูกต้องสูงสุด) |



## Part 2: The Bottleneck — Stack vs Heap
- **Stack**: ตัวเลข/ตัวแปรเล็ก + ตัวชี้ (reference) ไปยังข้อมูลจริง
- **Heap**: เก็บวัตถุจริง (list, DataFrame) กินพื้นที่เยอะ
- **Garbage Collector**: เก็บของใน Heap ที่ไม่มี reference ชี้ถึง
- Analogy: รีโมท (stack) ↔ ทีวี (heap)

![Stack vs Heap](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/stack_vs_heap.png?raw=1)



### Copy vs View (ทำไม RAM พุ่ง)
- pandas/numpy บางครั้งให้ **view** (ชี้ข้อมูลเดิม) → ประหยัด RAM
- แต่บางครั้งสร้าง **copy** จริง → RAM ใช้เพิ่มทันที

![Reference vs Copy](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/reference_vs_copy.png?raw=1)
- รู้ทันด้วย `.copy()` และสังเกต memory usage



In [7]:
import numpy as np
arr = np.arange(10)
view = arr[2:6]   # view
view[0] = 999
print('arr:', arr)  # เปลี่ยนตามเพราะแชร์ memory

# copy = arr[2:6].copy()
# copy[0] = 555
# print('arr after copy change:', arr)



arr: [  0   1 999   3   4   5   6   7   8   9]


### Garbage Collection (GC) & Memory Leak — สิ่งที่ Big Data Engineer ต้องรู้

**1. Garbage Collection (GC)**:
- คือ "พนักงานทำความสะอาด" อัตโนมัติ (ใน Java/Python) ที่คอยเดินเก็บกวาดหน่วยความจำ (RAM) ส่วนที่ไม่มีใครใช้แล้วคืนให้ระบบ
- ถ้าไม่มี GC: โปรแกรมจะกิน RAM ไปเรื่อยๆ จนเครื่องค้าง

**2. Memory Leak**:
- คือการที่เรา "เผลอผูกเชือก" (Reference) ไว้กับขยะ ทำให้ GC ไม่กล้าเก็บไปทิ้ง เพราะนึกว่าเรายังใช้อยู่
- **ผลกระทบใน Big Data**: Spark Driver/Executor มักจะตายด้วย error `OutOfMemory` เพราะ Code ของเราเขียน dataframe เก็บใส่ list ไว้เรื่อยๆ โดยไม่เคลียร์ทิ้ง

![GC & Memory Leak](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/gc_memory_leak.png?raw=1)



In [ ]:
# Demo: reference vs copy
import sys, copy

a = [1, 2, 3]
b = a              # แชร์รีโมท
c = copy.deepcopy(a)  # สำเนาของจริง

print('id(a)', id(a), 'id(b)', id(b), 'id(c)', id(c))
a.append(99)
print('หลัง append ที่ a:')
print('a =', a)
print('b =', b, '<-- เปลี่ยนตามเพราะแชร์ reference')
print('c =', c, '<-- ไม่เปลี่ยนเพราะ copy ของจริง')
print('ขนาด list a (bytes):', sys.getsizeof(a))



id(a) 4685899840 id(b) 4685899840 id(c) 4685897216
หลัง append ที่ a:
a = [1, 2, 3, 99]
b = [1, 2, 3, 99] <-- เปลี่ยนตามเพราะแชร์ reference
c = [1, 2, 3] <-- ไม่เปลี่ยนเพราะ copy ของจริง
ขนาด list a (bytes): 88


### Intro to Pandas & DataFrame (ปูพื้นฐาน)
- **Pandas**: คือ Library สามัญประจำบ้านของ Python สำหรับจัดการข้อมูลตาราง (เหมือน Excel ติดปีก)
- **DataFrame**: คือ Object หลักของ Pandas หน้าตาเหมือนตาราง มี Row Index และ Column Name
- แต่ระวัง! DataFrame โหลดข้อมูลทุกอย่างลง RAM (In-memory) ถ้าไฟล์ใหญ่กว่า RAM เครื่อง -> จบเห่ (ต้องใช้ Spark/Dask แทน)

### Memory Footprint ของ DataFrame — เลือก dtype ให้เหมาะ
- ค่า `int64/float64` หนักกว่า `int32/float32`
- หมวดหมู่ซ้ำควรใช้ `category` → ลด RAM และทำงานกับ Parquet dictionary ได้ดี
- ชอบตั้งคำถาม: "คอลัมน์นี้จำเป็นต้องเป็นอะไรยาวสุดไหม?" เช่น `user_id` 0-1e6 พอใช้ `int32`



In [8]:
import pandas as pd, numpy as np
rows = 200_000
raw_df = pd.DataFrame({
    'user_id': np.random.randint(0, 1_000_000, rows),
    'country': np.random.choice(['TH','US','JP','DE','FR','UK'], rows),
    'value': np.random.randn(rows)
})

print('default dtypes:', raw_df.dtypes)
print('memory (MB):', raw_df.memory_usage(deep=True).sum() / 1e6)

optimized = raw_df.assign(
    user_id = raw_df['user_id'].astype('int32'),
    country = raw_df['country'].astype('category'),
    value = raw_df['value'].astype('float32')
)
print('\noptimized dtypes:', optimized.dtypes)
print('memory (MB):', optimized.memory_usage(deep=True).sum() / 1e6)



default dtypes: user_id      int64
country     object
value      float64
dtype: object
memory (MB): 13.400132

optimized dtypes: user_id       int32
country    category
value       float32
dtype: object
memory (MB): 1.80061


### Data Architecture Evolution: Warehouse vs Lake vs Lakehouse

1. **Data Warehouse (ยุค 1990s)**:
   - เก็บข้อมูลมีโครงสร้าง (Table)
   - ข้อดี: เป็นระเบียบ, เร็ว, เชื่อถือได้
   - ข้อเสีย: แพง, ไม่รองรับไฟล์ภาพ/เสียง/Video

2. **Data Lake (ยุค 2010s)**:
   - ทะเลข้อมูล เททุกอย่างลงไป (Files)
   - ข้อดี: ถูก, เก็บได้ทุกท่า
   - ข้อเสีย: เละเทะ (Data Swamp), ไม่รองรับ Transaction (ACID), Update ยาก

3. **Data Lakehouse (ยุค 2020s)**:
   - ลูกผสม: เก็บเป็นไฟล์ใน Lake (ถูก) + มี Metadata Log คุม (ACID, Schema)
   - ทำให้เราสามารถ `UPDATE/DELETE` ข้อมูลในไฟล์ Parquet ได้เหมือนทำใน Database!

| คุณสมบัติ | Warehouse | Lake | Lakehouse (Delta/Iceberg/Hudi) |
|---|---|---|---|
| Schema enforcement | สูง | ต่ำ | ปานกลาง/สูง |
| ACID | ✅ | ❌ | ✅ (ผ่าน log + manifest) |
| Format หลัก | Proprietary | Any File | Open Format (Parquet) + Log |



## Part 3: Modern Storage Wars — Hands-on
เปรียบเทียบ CSV (row-based) vs Parquet (columnar)



### Format Zoo — CSV/JSON/Avro/Parquet/ORC
| Format | โครงสร้าง | จุดเด่น | จุดอ่อน |
|---|---|---|---|
| CSV | Row | อ่านง่าย, ใช้ได้ทุกที่ | ไม่มี schema, ใหญ่, ช้า |
| JSON | Row/Nested | ยืดหยุ่น, nested ได้ | ใหญ่, parsing ช้า |
| Avro | Row + Schema | schema ชัด, write fast | ไม่เหมาะสำหรับ column pruning |
| Parquet | Columnar | อ่านเร็ว, compression ดี | write ช้ากว่า row format |
| ORC | Columnar | สถิติละเอียด, compact | tooling น้อยกว่า Parquet |



### Row vs Columnar — เลือกเก็บอย่างไร
- **Row (CSV/JSON)**: เขียน/อ่านทั้งแถวรวดเดียว → เหมาะกับ workload เขียนต่อเนื่องหรืออ่านทุกคอลัมน์
- **Columnar (Parquet/ORC)**: จัดเก็บทีละคอลัมน์ → อ่านเฉพาะคอลัมน์ที่ใช้ (column pruning), บีบอัดได้สูง, รองรับ predicate pushdown
- จุดตัดสินใจ: ถ้า query ส่วนใหญ่เลือกไม่กี่คอลัมน์หรือมี filter ชัดเจน → columnar ชนะ; ถ้าต้อง append record แบบ real-time และอ่านทุกคอลัมน์ → row-format อาจพอเพียง

![Row vs Columnar](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/row_vs_columnar.png?raw=1)



### ตัวอย่าง Column Pruning แบบภาพจำ
มีตาราง 5 คอลัมน์ แต่ query ใช้แค่ 2 คอลัมน์: columnar จะอ่านเฉพาะ 2 คอลัมน์นั้น
→ ประหยัด I/O และเวลามาก โดยเฉพาะเมื่อตารางมีหลายคอลัมน์

![Column Pruning](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/column_pruning.png?raw=1)



In [ ]:
import pandas as pd
import numpy as np
import os, time, duckdb
np.random.seed(42)



In [11]:
# สร้างข้อมูลจำลอง

def generate_data(rows=1_000_000):
    df = pd.DataFrame({
        'id': np.arange(rows),
        'category': np.random.choice(['Electronic', 'Clothing', 'Furniture', 'Food'], rows),
        'status': np.random.choice(['Completed', 'Pending', 'Failed'], rows),
        'value': np.random.randn(rows),
        'description': np.random.choice(['Data for big data class'] * 5, rows)
    })
    return df

%time df = generate_data()
df.head()



CPU times: user 451 ms, sys: 245 ms, total: 697 ms
Wall time: 696 ms


,id,category,status,value,description
0,0,Furniture,Pending,-1.586524,Data for big data class
1,1,Food,Completed,0.801666,Data for big data class
2,2,Electronic,Pending,-1.476044,Data for big data class
3,3,Food,Pending,-0.222654,Data for big data class
4,4,Clothing,Pending,-0.907878,Data for big data class


In [12]:
# Save & เปรียบเทียบขนาดไฟล์
%time df.to_csv('data.csv', index=False)
%time df.to_parquet('data.parquet', index=False)

csv_mb = os.path.getsize('data.csv') / (1024 * 1024)
parquet_mb = os.path.getsize('data.parquet') / (1024 * 1024)

print(f"CSV Size: {csv_mb:.2f} MB")
print(f"Parquet Size: {parquet_mb:.2f} MB")
print(f"เล็กลงกี่เท่า: {csv_mb / parquet_mb:.1f}x")



CPU times: user 3.98 s, sys: 75.3 ms, total: 4.06 s
Wall time: 4.07 s
CPU times: user 429 ms, sys: 29 ms, total: 458 ms
Wall time: 457 ms
CSV Size: 64.47 MB
Parquet Size: 12.46 MB
เล็กลงกี่เท่า: 5.2x


### Read Only Needed Columns (pyarrow)
อ่านเฉพาะคอลัมน์ที่ใช้จริง → เร็วขึ้น และใช้ RAM น้อยลง



In [ ]:
import pyarrow.parquet as pq
pf = pq.ParquetFile('data.parquet')
# อ่านเฉพาะคอลัมน์ที่ต้องใช้
subset = pf.read(columns=['status', 'value']).to_pandas()
subset.head()



### Row Group Size Experiment (Deep Dive)
Row Group คือการแบ่งไฟล์ Parquet เป็นก้อนย่อย ๆ ในแนวนอน (Horizontal Partitioning ภายในไฟล์)

**Trade-off ที่ต้องรู้:**
1. **Row Group ใหญ่ (เช่น 512MB - 1GB):**
   - ✅ Compression ดีขึ้นมาก (เห็น pattern ข้อมูลเยอะขึ้น)
   - ✅ Metadata น้อยลง (File footer เล็ก)
   - ❌ อ่านช้าถ้าเราต้องการแค่ "บางแถว" (ต้องโหลดมาทั้งก้อนเพื่อหาแถวที่ต้องการ)

2. **Row Group เล็ก (เช่น 1MB - 10MB):**
   - ✅ Pruning แม่นยำ (ข้ามก้อนที่ไม่เกี่ยวได้เยอะ)
   - ❌ Metadata บวม (Header/Footer กินที่)
   - ❌ Compression แย่ลง

**Best Practice:** ใช้ default (~128MB) หรือปรับตามลักษณะการ Query


In [ ]:
import pyarrow as pa, pyarrow.parquet as pq, os, time

table = pa.Table.from_pandas(df)
# ลอง save เปรียบเทียบขนาดระหว่าง Row Group เล็ก vs ใหญ่
for rg in [5000, 1_000_000]:
    fn = f"rg_{rg}.parquet"
    pq.write_table(table, fn, row_group_size=rg)
    size_mb = os.path.getsize(fn)/(1024*1024)
    print(f"Row Group Size: {rg:<10} File Size: {size_mb:.2f} MB")


### Compression Shootout (Snappy vs Gzip vs Zstd)
ลองบีบอัด Parquet ด้วย codec ต่าง ๆ แล้วดูขนาด/เวลาเขียน
- Snappy: เร็ว, ขนาดพอประมาณ (เหมาะ default)
- Gzip: ช้ากว่า แต่เล็กลงอีก
- Zstd: สมดุล ขนาดเล็กกว่า Snappy แต่เร็วกว่า Gzip



In [ ]:
import time, os
import pandas as pd

codecs = ['snappy','gzip','zstd','none']
rows = 400_000
cdf = df.sample(rows, replace=False).reset_index(drop=True)

results = []
for codec in codecs:
    fn = f"data_{codec}.parquet"
    start = time.time()
    cdf.to_parquet(fn, compression=None if codec=='none' else codec, index=False)
    elapsed = time.time() - start
    size_mb = os.path.getsize(fn)/(1024*1024)
    results.append((codec, size_mb, elapsed))

pd.DataFrame(results, columns=['codec','size_mb','write_sec']).sort_values('size_mb')



### Parquet Internals แบบสั้นๆ
- ไฟล์ถูกแบ่งเป็น **Row Groups** (มัก ~128MB) → เป็นหน่วยที่ถูกอ่าน/ข้ามได้
- แต่ละคอลัมน์ใน row group มี **Dictionary** (ถ้าเปิด) + **min/max stats** + optional **Bloom filter**
- เวลา query: engine ดู metadata ก่อน → ตัด row group ที่ค่า min/max ไม่ทับกับเงื่อนไข (row-group pruning) แล้วค่อยอ่านเฉพาะคอลัมน์ที่ใช้
- Stats ยิ่งแม่นเมื่อข้อมูลถูก sort/cluster ตามคอลัมน์ที่เรามัก filter

![Parquet Structure](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/parquet_structure.png?raw=1)



### ทำไม Parquet ถึงเล็กและอ่านเร็ว?
- **Dictionary Encoding**: เก็บตาราง mapping แล้วเก็บเป็นตัวเลขสั้น ๆ
- **Run-Length Encoding (RLE)**: ค่าซ้ำกันยาว ๆ เก็บเป็น “ค่า × จำนวนครั้ง”
- **Columnar Layout**: เก็บทีละคอลัมน์ → อ่านเฉพาะคอลัมน์ที่ต้องใช้ (column pruning)
- **Predicate Pushdown**: อ่านเฉพาะ row group ที่เงื่อนไขตรง (เช่น status = 'Completed')

![Encoding Techniques](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/encoding_techniques.png?raw=1)



In [13]:
# Demo: column pruning & predicate pushdown ด้วย DuckDB
# DuckDB จะอ่านไฟล์โดยตรง (ไม่ต้อง load ทั้งไฟล์เข้าหน่วยความจำก่อน)

# อ่านเฉพาะ 2 คอลัมน์
%time duckdb.query("SELECT category, status FROM 'data.csv' LIMIT 5").df()
%time duckdb.query("SELECT category, status FROM 'data.parquet' LIMIT 5").df()

# นับจำนวนสถานะ Completed (predicate pushdown ทำงานกับ Parquet)
%time duckdb.query("SELECT count(*) FROM 'data.csv' WHERE status='Completed'").df()
%time duckdb.query("SELECT count(*) FROM 'data.parquet' WHERE status='Completed'").df()



CPU times: user 177 ms, sys: 59.2 ms, total: 236 ms
Wall time: 473 ms
CPU times: user 7.18 ms, sys: 1.03 ms, total: 8.21 ms
Wall time: 7.19 ms
CPU times: user 489 ms, sys: 39.7 ms, total: 529 ms
Wall time: 344 ms
CPU times: user 12.1 ms, sys: 0 ns, total: 12.1 ms
Wall time: 12.7 ms


,count_star()
0,333421


In [29]:
data = pd.read_parquet('data.parquet')

In [31]:
# 1. ทดสอบอ่านแค่คอลัมน์เดียว (Column Pruning)
# เปลี่ยนจาก SELECT category, status เป็น SELECT status อย่างเดียว

print("--- Reading 1 Column (status) ---")
print("CSV Time:")
%time duckdb.query("SELECT status FROM 'data.csv' LIMIT 5").df()

print("\nParquet Time:")
%time duckdb.query("SELECT status FROM 'data.parquet' LIMIT 5").df()

# 2. ทดสอบนับจำนวน (Predicate Pushdown) โดยใช้แค่คอลัมน์เดียว
print("\n--- Counting Specific Status (Predicate Pushdown) ---")
print("CSV Time:")
%time duckdb.query("SELECT count(status) FROM 'data.csv' WHERE status='Completed'").df()

print("\nParquet Time:")
%time duckdb.query("SELECT count(status) FROM 'data.parquet' WHERE status='Completed'").df()

--- Reading 1 Column (status) ---
CSV Time:
CPU times: user 161 ms, sys: 35.7 ms, total: 197 ms
Wall time: 212 ms

Parquet Time:
CPU times: user 2.99 ms, sys: 0 ns, total: 2.99 ms
Wall time: 4.64 ms

--- Counting Specific Status (Predicate Pushdown) ---
CSV Time:
CPU times: user 574 ms, sys: 44.9 ms, total: 619 ms
Wall time: 619 ms

Parquet Time:
CPU times: user 23.9 ms, sys: 989 µs, total: 24.8 ms
Wall time: 25.1 ms


,count(status)
0,333421


### Read Speed Compare — DuckDB vs pandas
- DuckDB อ่านจากไฟล์โดยตรง ใช้ predicate/column pruning เต็มที่
- pandas ต้องโหลดทั้งไฟล์ (แต่กับ Parquet ยังได้ column pruning บางส่วน)



In [27]:
import pandas as pd, time

start = time.time(); pd.read_csv('data.csv', usecols=['status']).query("status=='Completed'").shape[0]; csv_time = time.time()-start
start = time.time(); pd.read_parquet('data.parquet', columns=['status']).query("status=='Completed'").shape[0]; pq_time = time.time()-start

print(f"pandas CSV count took {csv_time:.2f}s; Parquet took {pq_time:.2f}s")



pandas CSV count took 0.59s; Parquet took 0.10s


In [14]:
# Inspect Parquet metadata (row groups + stats) ด้วย PyArrow
import pyarrow.parquet as pq
pf = pq.ParquetFile('data.parquet')
print('Row groups:', pf.num_row_groups)
print('Schema:')
print(pf.schema)

# ดูสถิติของคอลัมน์ status ใน row group แรก
rg0 = pf.metadata.row_group(0).column(pf.schema.names.index('status'))
print('status stats rowgroup0:', rg0.statistics)



Row groups: 1
Schema:
required group field_id=-1 schema {
  optional int64 field_id=-1 id;
  optional binary field_id=-1 category (String);
  optional binary field_id=-1 status (String);
  optional double field_id=-1 value;
  optional binary field_id=-1 description (String);
}

status stats rowgroup0: <pyarrow._parquet.Statistics object at 0x7d2c64dc6840>
  has_min_max: True
  min: Completed
  max: Pending
  null_count: 0
  distinct_count: None
  num_values: 1000000
  physical_type: BYTE_ARRAY
  logical_type: String
  converted_type (legacy): UTF8


### Data Quality Check Mini-lab
- ตรวจ missing values / duplicate IDs
- คำนวณ % missing แล้วตัดสินใจว่าจะลบ เติม หรือเก็บไว้



In [ ]:
# ตรวจข้อมูลคุณภาพเบื้องต้น
print('missing per column:')
print(df.isna().mean())
print('duplicate id count:', df['id'].duplicated().sum())



### Activity 2 — Cardinality & Compression
ลองปรับจำนวน category/status ให้หลากหลายขึ้น แล้ววัดขนาด Parquet เทียบ CSV
- คาดหวัง: เมื่อค่าซ้ำลดลง (cardinality สูง) ประสิทธิภาพการบีบอัดจะลดลง
- ให้ลอง rows = 200_000, 1_000_000 และ cardinality ต่าง ๆ



#### Partitioning: ทำแบบไหนดี?
- ดี: `year/month/day` สำหรับข้อมูลตามเวลา → filter ง่าย
- ระวัง: partition ตาม `user_id` หรือ `country` ที่มีค่ากระจายมาก → ไฟล์เล็กนับหมื่น
- ใช้ `repartition` เพื่อคุมจำนวนไฟล์ เป้าหมายขนาด 128–512MB/ไฟล์
- หลัง append หลายรอบควรมีขั้นตอน **compaction** รวมไฟล์



In [17]:
# Experiment: cardinality vs size (Parquet vs CSV)
import pandas as pd, numpy as np, os, time

rows = 300_000
cardinalities = [4, 20, 200, 1000]
results = []

for card in cardinalities:
    categories = [f"cat_{i}" for i in range(card)]
    df = pd.DataFrame({
        'id': np.arange(rows),
        'cat': np.random.choice(categories, rows),
        'value': np.random.randn(rows)
    })
    csv_path = f"card_{card}.csv"
    pq_path = f"card_{card}.parquet"
    df.to_csv(csv_path, index=False)
    df.to_parquet(pq_path, index=False)
    csv_mb = os.path.getsize(csv_path)/(1024*1024)
    pq_mb = os.path.getsize(pq_path)/(1024*1024)
    results.append((card, csv_mb, pq_mb, csv_mb/pq_mb))

pd.DataFrame(results, columns=['cardinality','csv_mb','parquet_mb','ratio_csv_div_parquet']).sort_values('cardinality')



,cardinality,csv_mb,parquet_mb,ratio_csv_div_parquet
0,4,9.229452,4.040214,2.284397
1,20,9.373487,4.147559,2.260001
2,200,9.644685,4.255451,2.266431
3,1000,9.771169,4.330169,2.256533


### Schema Evolution Mini-lab
- เพิ่มคอลัมน์ใหม่ (เช่น `discount`) แล้วเขียนทับในโฟลเดอร์ Parquet เดิม
- สังเกตว่าไฟล์ใหม่มี schema ต่างจากไฟล์เก่า → query engine ต้อง union schema
- DuckDB / Spark รองรับการ union schema อัตโนมัติถ้าตั้งค่าเปิด



In [3]:
# เพิ่มคอลัมน์ใหม่และเขียนทับบาง partition
import pandas as pd
new_df = df.copy()
new_df['discount'] = np.random.randint(0, 30, len(new_df))

duckdb.query("COPY (SELECT * FROM new_df) TO 'lakehouse' (FORMAT 'parquet', PARTITION_BY ['cat'])")

# อ่านรวม schema
duckdb.query("SELECT count(*), avg(discount) FROM 'lakehouse'").df().head()



NameError: name 'df' is not defined

### Quick Check (ระหว่างคาบ)
1) ทำไม NameNode/S3 list หลายพันไฟล์ถึงช้า แม้ไฟล์จะเล็กมาก?  
2) ถ้า query ส่วนใหญ่เลือกแค่ 3 คอลัมน์จาก 200 คอลัมน์ คุณจะเลือกฟอร์แมตอะไร? ทำไม?  
3) จะตั้ง partition อย่างไรสำหรับตาราง log ต่อวันให้ query `WHERE date BETWEEN ...` เร็ว แต่ไม่ระเบิดจำนวนไฟล์?



### Activity 3 — Mini Lakehouse (Partition & Query)
1. สร้างโฟลเดอร์ `lakehouse/` และเขียน Parquet แบบ partition ตาม `status`
2. ใช้ DuckDB อ่านจากโฟลเดอร์นั้น เช่น `SELECT status, COUNT(*) FROM 'lakehouse' GROUP BY 1`
3. ลอง query เฉพาะ `status='Failed'` เพื่อดูเวลา (partition pruning)



In [21]:
# Build mini lakehouse partitioned by status
import os, duckdb
import pyarrow.parquet as pq

os.makedirs('lakehouse', exist_ok=True)
# เขียนแบบ partition
duckdb.query("COPY (SELECT * FROM df) TO 'lakehouse' (FORMAT 'parquet', PARTITION_BY ['status'])")

# Query ทั้งก้อน
%time duckdb.query("SELECT status, COUNT(*) AS cnt, AVG(value) AS avg_value FROM 'lakehouse' GROUP BY 1").df()

# Query เฉพาะ Failed (จะ prune partition อื่น)
%time duckdb.query("SELECT * FROM 'lakehouse' WHERE status='Failed' LIMIT 5").df()



BinderException: Binder Error: "partition_by" expected to find [status], but it was not found in the table

### 💥 Experiment: The "Small Files" Nightmare (Bad Partitioning)
ห้าม Partition ด้วยคอลัมน์ที่มีค่าไม่ซ้ำกันเยอะ ๆ (High Cardinality) เช่น `id` หรือ `timestamp` ละเอียดเกินไป
เพราะจะทำให้เกิดไฟล์ขยะจำนวนมหาศาล (Small Files Problem) ที่ทำให้ระบบล่มได้


In [22]:
# Demo: หายนะของการ Partition ผิด (อย่าหาทำใน Production!)
import shutil
import duckdb

# สร้าง high cardinality dataframe
bad_df = df.copy()
bad_df['many_partitions'] = bad_df['id'] % 2000  # สร้าง 2,000 partitions

output_dir = 'bad_partitioning'
if os.path.exists(output_dir): shutil.rmtree(output_dir)
os.makedirs(output_dir)

print('Starting bad partition write...')
%time duckdb.query("COPY (SELECT * FROM bad_df) TO 'bad_partitioning' (FORMAT 'parquet', PARTITION_BY ['many_partitions'])")

num_files = sum([len(files) for r, d, files in os.walk(output_dir)])
print(f"สร้างไฟล์ไปทั้งหมด: {num_files} ไฟล์!")
# Cleanup
if os.path.exists(output_dir): shutil.rmtree(output_dir)


Starting bad partition write...


BinderException: Binder Error: "partition_by" expected to find [many_partitions], but it was not found in the table

สร้างไฟล์ไปทั้งหมด: 0 ไฟล์!


In [35]:
%time duckdb.query("SELECT category FROM 'data.parquet'").df()

CPU times: user 65.4 ms, sys: 40.9 ms, total: 106 ms
Wall time: 106 ms


,category
0,Furniture
1,Food
2,Electronic
3,Food
4,Clothing
...,...
999995,Furniture
999996,Clothing
999997,Electronic
999998,Electronic


### Lab Report Template (ส่งท้ายคาบ)
ให้สรุปผลการทดลองเป็น 1 หน้าสั้น ๆ:
- ขนาดไฟล์ CSV vs Parquet (กี่เท่า?)
- เวลาอ่านคอลัมน์เดียว vs ทั้งตาราง
- วิธี partition ที่เลือกและเหตุผล
- ข้อสังเกตเรื่อง small files หรือ row group size



## Wrap-up & Post-test
**Takeaways**
1. File system มี block/fragmentation → small files problem
2. DB ชนะไฟล์ตรงที่ ACID + Index
3. Stack = รีโมท, Heap = ทีวี, GC เก็บของที่ไม่มีรีโมทชี้
4. Parquet/Columnar = เล็กและเร็วด้วย dictionary + RLE + column pruning + predicate pushdown
5. Lakehouse = Parquet + Transaction Log (Delta/Iceberg/Hudi)




### After Class — อ่านต่อถ้ามีเวลา
- DuckDB Docs: Query Parquet & metadata, filter/column pushdown
- AWS EMR/Athena Best Practices: ขนาดไฟล์ 128–512MB, หลีกเลี่ยง partition ลึก
- Blog: Parquet predicate pushdown (row-group pruning, dictionary, bloom filter)
- HDFS Small Files Problem: ทำไม metadata จึงล่มง่าย และวิธีแก้ด้วย compaction


---
# 📝 Lab Report Template (ส่งท้ายคาบ)
**ชื่อ-นามสกุล:** ปรวัฒน์ ไชยหอม
**รหัสนักศึกษา:** 6704800023
### Part 1: Experimental Results
บันทึกผลการทดลองจาก Code ที่รันใน Lab

| การทดลอง | ผลลัพธ์ (ตัวเลข/หน่วย) |
|---|---|
| 1. ขนาดไฟล์ CSV (MB) | 64.47 |
| 2. ขนาดไฟล์ Parquet (MB) | 12.46 |
| 3. Parquet เล็กลงกี่เท่า? (CSV / Parquet) | 5.2 |
| 4. เวลาอ่าน CSV ทั้งหมด (วินาที) | 0.59 |
| 5. เวลาอ่าน Parquet ทั้งหมด (วินาที) | 0.10 |
| 6. เวลาอ่าน Parquet แค่ 1 คอลัมน์ (วินาที) | 106 |

### Part 2: Critical Thinking
**1. Partitioning Strategy:**
จากข้อมูล Log รายวัน คุณจะเลือก Partition ด้วยอะไร? ระหว่าง `date` vs `user_id`? เพราะเหตุใด?
>
ตอบ: เลือก date เพราะข้อมูล Log มีลักษณะอิงตามลำดับเวลา การ Partition ตามวันที่ช่วยให้การค้นหาข้อมูลทำได้รวดเร็วและยังช่วยให้การบริหารจัดการข้อมูลทำได้ง่าย

**2. Small Files Problem:**
ถ้าเรา Save ข้อมูลทุกๆ 1 วินาทีลง Data Lake จะเกิดปัญหาอะไรขึ้น? และควรแก้อย่างไร?
>
ตอบ: ปัญหา: เกิด Metadata Overhead ทำให้ระบบไฟล์ทำงานหนัก ทำให้เสียเวลาในการเปิด-ปิดไฟล์จำนวนมาก
วิธีแก้: ทำการรวบรวมข้อมูลเป็นรอบ ก่อนเขียนลงดิสก์ หรือรัน Job เพื่อรวมไฟล์เล็กให้เป็นไฟล์ขนาดใหญ่ อย่างสม่ำเสมอ



### Part 3: Code Challenge (เติมคำในช่องว่าง)
จงเติม Code ให้สมบูรณ์เพื่ออ่านเขียนไฟล์ Parquet อย่างมีประสิทธิภาพ



In [36]:
# 1. อ่านไฟล์ Parquet โดยเลือกโหลดเฉพาะคอลัมน์ 'status' และ 'value' เพื่อประหยัด RAM
df = pd.read_parquet('data.parquet', columns=['_______', '_______'])

# 2. เขียนไฟล์ Parquet โดยใช้การบีบอัดแบบ 'zstd' เพื่อให้ขนาดเล็กที่สุด
df.to_parquet('output.parquet', compression='_______')

# 3. ใช้ DuckDB เขียนข้อมูลลงโฟลเดอร์โดยแบ่ง Partition ตามคอลัมน์ 'date'
duckdb.query("COPY (SELECT * FROM my_table) TO 'my_data' (FORMAT 'parquet', PARTITION_BY ['_______'])")

# 4. เช็คขนาดไฟล์ (byte) ของไฟล์ที่ชื่อ data.parquet
file_size = os.path._______('data.parquet')

# 5. (Concept) ถ้าเราต้องการอ่านเฉพาะ Row Group ที่มี status='Completed' ระบบใช้เทคนิคอะไรในการกระโดดข้าม?
# ตอบ: ___________________ (Predicate Pushdown / Indexing / Caching)


ArrowInvalid: No match for FieldRef.Name(_______) in id: int64
category: string
status: string
value: double
description: string
__fragment_index: int32
__batch_index: int32
__last_in_fragment: bool
__filename: string